In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import pydicom
#ds = pydicom.dcmread('/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train/063319de25ce7edb9b1c6b8881290140.dicom')
#plt.imshow(ds.pixel_array, cmap=plt.cm.bone) 
plt.imshow(result_list[0], cmap=plt.cm.bone) 

In [ ]:
import vtk
from vtk.util import numpy_support
import numpy

PathDicom = "/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train/"
reader = vtk.vtkDICOMImageReader()
reader.SetDirectoryName(PathDicom)
reader.Update()
# Load dimensions using `GetDataExtent`
_extent = reader.GetDataExtent()
ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]

# Load spacing values
ConstPixelSpacing = reader.GetPixelSpacing()
# Get the 'vtkImageData' object from the reader
imageData = reader.GetOutput()
# Get the 'vtkPointData' object from the 'vtkImageData' object
pointData = imageData.GetPointData()
# Ensure that only one array exists within the 'vtkPointData' object
#assert (pointData.GetNumberOfArrays()==1)
# Get the `vtkArray` (or whatever derived type) which is needed for the `numpy_support.vtk_to_numpy` function
arrayData = pointData.GetArray(1)

# Convert the `vtkArray` to a NumPy array
ArrayDicom = numpy_support.vtk_to_numpy(arrayData)
# Reshape the NumPy array to 3D using 'ConstPixelDims' as a 'shape'
ArrayDicom = ArrayDicom.reshape(ConstPixelDims, order='F')
ArrayDicom

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from vtk.numpy_interface import dataset_adapter as dsa


a = dsa.WrapDataObject(imageData)
print(a.PointData.keys())
vt = a.PointData['DICOMImage']
vt[0]


In [ ]:
import pandas as pd

df = pd.read_csv('/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train.csv')
df

In [ ]:
df['class_name'].value_counts()

In [ ]:
foldertrain = '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train/'

df['folder_train'] = '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train/' + df['image_id'] + '.dicom'

df

In [ ]:
!pip install dicom_parser

In [ ]:
from dicom_parser import Image
image = Image('/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train/063319de25ce7edb9b1c6b8881290140.dicom')
#for k,v in image.header.raw.items():
    #print(k,v)
#image.header.raw[list(image.header.raw.keys())[-1]].value
image._data

In [ ]:
from pydicom import dcmread
from dicom_parser import Image

df2 = df[:10000]
df2['folder_train'].count()
df2['folder_train_processed'] = df2['folder_train'].map(lambda x : Image(x))
df2

In [ ]:
from pydicom import dcmread
import multiprocess as mp
from pydicom import dcmread
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import numpy as np

def dicom2array(path):
    voi_lut=True
    fix_monochrome=True
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

result_list = []
def log_result(result):
    # This is called whenever foo_pool(i) returns a result.
    # result_list is modified only by the main process, not the pool workers.
    result_list.append(result)


df2 = df[:1001]
li_to_proc = df2['folder_train'].to_list()
print(len(li_to_proc))
pool = mp.Pool(processes=5)#,maxtasksperchild=1000)
dcms_info = [pool.apply_async(dicom2array, args=(dcm_path,), callback = log_result) for dcm_path in li_to_proc]

pool.close()
pool.join()


In [ ]:
len(result_list)

In [ ]:
xx= [[0,1001]]
k = [ [xx[0][1]*x-(x)+1, xx[0][1]*(x+1)-(x)] for x in range(10) ]
for xk in k:
    print(xk)

In [ ]:
from pydicom import dcmread
import multiprocess as mp
from pydicom import dcmread
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import numpy as np
import os
import gc
import psutil
import pickle

def dicom2array(path):
    voi_lut=True
    fix_monochrome=True
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

result_list = []
def log_result(result):
    # This is called whenever foo_pool(i) returns a result.
    # result_list is modified only by the main process, not the pool workers.
    result_list.append(result)
    
xx= [[0,500]]
k = [ [xx[0][1]*x-(x)+1, xx[0][1]*(x+1)-(x)] for x in range(10) ]
for xk in k:
    df2 = df[xk[0]:xk[1]]
    li_to_proc = df2['folder_train'].to_list()
    print(len(li_to_proc))
    pool = mp.Pool(processes=5,maxtasksperchild=10)
    dcms_info = [pool.apply_async(dicom2array, args=(dcm_path,), callback = log_result) for dcm_path in li_to_proc] 

    pool.close()
    pool.join()
    pool.terminate()
    del pool
    proc = psutil.Process(os.getpid())
    gc.collect()
    with open('l1.pkl', 'wb') as f:
        pickle.dump(result_list, f)
   



In [ ]:
import tensorflow as tf 

def dicom2array(path):
    voi_lut=True
    fix_monochrome=True
    print(path)
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

filenames_dataset = tf.data.Dataset.list_files(df['folder_train'].to_list())
image_dataset = filenames_dataset.map(dicom2array)

In [ ]:
xx= [[0,500]]
k = [ [xx[0][1]*x-(x)+1, xx[0][1]*(x+1)-(x)] for x in range(10) ]
k

In [ ]:
import tensorflow as tf
batch_size = 32
img_height = 224
img_width = 224



train_list_ds = tf.data.Dataset.from_tensor_slices((total_x, total))


train_list_ds = tf.keras.preprocessing.image_dataset_from_directory(
  './',
  validation_split=0.3,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  './',
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)